# Data Preparation

The data that we will use throughout this research is primarily comprised of tweets and the full body text from wesbite articles. The two websites in question are [Amren](https://www.amren.com/) and [Dailystormer](https://www.dailystormer.com). NB. The Dailystormer website was taken off the internet in August of 2017.

I started using MongoDB to store my tweets at the start of this research. More specifically, I used [Twitter Goblin](https://github.com/JherezTaylor/twitter-goblin), a service for handling the Twitter streaming and piping data directly to a MongoDB install. I mention this because I modified the tweet object when I stored them, I removed fields that I did not need, flattened the nested tweet object, deleted retweets and cleaned up several other fields. I did this before moving to ElasticSearch in order save space and time. So in summary, if you use your own data or if you use the data dump that I include, it is important to be aware that the tweet structure will not be the same as what the API describes.

It is possible to replicate the main dataset of tweets by crawling the streaming API with the list in `data/search_streaming/refined_hs_keywords`. As mentioned in the readme, this list came from the [Hatebase Organization](https://www.hatebase.org). For the dataset storing the website articles and the tweets of users who shared those articles, you can collect them here:

- [Website article datasets](https://www.dropbox.com/s/lcg2j3zx2kuz2re/dailystormer_archive.20170901.gz?dl=0)
- [User Tweets datasets](https://www.dropbox.com/s/96mcbq260mgo1gs/melvyn_hs_users.20170901.gz?dl=0)

Restore the either dataset as follows:

```bash
nohup mongorestore --archive=dailystormer_archive.20170901.gz --gzip --db dailystormer -v --noIndexRestore --numInsertionWorkersPerCollection 5 &
```

With that out the way, if we are only concerned with the text field then we can continue.

## Tweet field cleaning

The logic for this can be found in `modules.preprocessing.db_cleaning` and it can be run from `hatespeech_core/core.py`

## Text Preprocessing

The first major task that needs to be done is text preprocessing. At this stage we create two types of preprocessed text. I do this in batches and with multiple threads using Spacy. The code is well documented and the starting point can be found in `modules.preprocessing.feature_prep` and run from `feature_prep.start_store_preprocessed_text()`

NB. I did extensive tests for my setup and determined that the optimal number of processor cores to use was 4. If you run this be sure to change the variable `num_cores` to suit your needs.

#### preprocessed_txt
This preserves emoji, hashtags and mentions. Standard tweet preprocessing with a battle tested preprocessor. The one I use here is provided by the NLP lab at Carnegie Mellon [Twokenizer](http://www.cs.cmu.edu/~ark/TweetNLP/). I modified a Python port of this, the original can be found [here.](https://github.com/myleott/ark-twokenize-py)

#### word_embedding_txt
This field prepares text for use in standard word embedding model such as word2vec or fasttext. It removes Removes URLS, numbers, and stopwords, normalizes @usermentions to `user_mention`. 


## Feature Extraction

What I do here is extract the individual tokens, stopwords, hashtags, and usermentions into their own fields for some data exploration. I also experimented with creating ngrams of dependency features. You can see the features by restoring the included datasets to a MongoDB setup. 

This stage is also responsible for extracting the Syntactic Dependency data needed for the dependency2vec. The data is in CoNLL-U format, for more details see the notebook `create_dep2vec`.

It is runs with `feature_prep.start_feature_extraction()`